<a href="https://colab.research.google.com/github/MamouneElBoukfaoui/California_house_value_prediction/blob/main/Binary_classification_using_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.datasets import imdb
(train_data, train_labels), (test_data,test_labels) = imdb.load_data(num_words=10000)

KeyboardInterrupt: 

Here we restrict oursleves to the top 10 000 most frequent words.
As shown below no word index is more than 10 000.

In [ ]:
max([max(sequence) for sequence in train_data])

# Data preparation

1. Decoding reviews


In [ ]:
word_index = imdb.get_word_index()
reverse_word_index = dict([(value,key) for (key,value) in word_index.items()])
decoded_review = ''.join([reverse_word_index.get(i-3, "?") for i in train_data[0]])

2. Encoding integer sequences via multi-hot encoding for neural network learning

In [ ]:
import numpy as np
def vectorize_sequences(sequences, dimension=10000):
  results = np.zeros((len(sequences),dimension))
  for i, sequence in enumerate(sequences):
    for j in sequence:
      results[i, j] = 1
  return results


x_train = vectorize_sequences(train_data)
x_test  = vectorize_sequences(test_data)

In [ ]:
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

# Building the model

1. Model architecture

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([layers.Dense(16, activation='relu')
                      ,layers.Dense(16, activation='relu'),
                          layers.Dense(1, activation = 'sigmoid')])



model.compile(optimizer='rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])


 2. Creating a validation set

In [ ]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

 3. Training the model

*train the model using 20 epochs on all training data using mini batches of 400 samples. We also measure the validation error.*

In [ ]:
history = model.fit(partial_x_train,partial_y_train,epochs=20, batch_size=512,
                    validation_data= (x_val,y_val))



In [ ]:
history_dict = history.history
history_dict.keys()

4. Plotting training and validation metrics

In [ ]:
import matplotlib.pyplot as plt
loss_val = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_val)+1)
plt.plot(epochs, loss_val, 'bo', label = 'Training loss ')
plt.plot(epochs, val_loss_values, 'b', label= 'Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('loss')
plt.legend()
plt.show()


In [ ]:
plt.clf()
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label = 'Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In the above two graphs, we observe that the validation loss seems to increase at the 4th epoch . Similarly, the validation acccuracy seems to reach a minimum at the 4th epoch. After the 4th epoch, the model seems to overfit the training data. Thus, we will train our model using 4 epochs

5. Training optimal model given by 4 epochs

In [ ]:
model_two = keras.Sequential([layers.Dense(16, activation='relu'), layers.Dense(16, activation = 'relu' ),
                              layers.Dense(1, activation='sigmoid')])

model_two.compile(optimizer= 'rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model_two.fit(x_train,y_train, epochs=4, batch_size=812)
results = model_two.evaluate(x_test,y_test)

# Generate predicitons on new data

In [ ]:
model_two.predict(x_test)[0][0]


The first review has a likelihood of 30% of being positive